# Создаем таблицу с информацией о видео на платформе

## Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Подготавливаем базовую таблицу

### Загружаем все необходимые таблицы

In [2]:
df_all = pd.read_csv('all_events.csv')

In [5]:
df_train = pd.read_csv('train_events.csv')

In [6]:
df_all.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,60,video_216518,10512324
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,121,video_41225,10951137
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585


In [7]:
df_train.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976


### Получаем таблицу со всеми записями 

In [8]:
df = pd.concat([df_all, df_train], axis = 0)

In [9]:
df.shape

(10199240, 9)

In [10]:
df.columns

Index(['event_timestamp', 'region', 'ua_device_type', 'ua_client_type',
       'ua_os', 'ua_client_name', 'total_watchtime', 'rutube_video_id',
       'viewer_uid'],
      dtype='object')

### Подгружаем и джойним таблицу с информацией о видео

In [11]:
df_info = pd.read_csv('video_info_v2.csv')

In [12]:
df_info.head()

,rutube_video_id,title,category,duration,author_id
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,1559160,1015054
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,1320007,1002180
2,video_476517,ПОПУЛЯРНЫЕ ВИДЕОИГРЫ в LEGO... перевод - TD BR...,Хобби,606145,1095337
3,video_157198,"Хороший лжец (фильм, 2019)",Фильмы,6577440,1043618
4,video_289824,Нашего старого гнобят по-всякому,Развлечения,859493,1009535


In [13]:
data = pd.merge(df, df_info, on = 'rutube_video_id', how = 'left')

In [14]:
data.columns

Index(['event_timestamp', 'region', 'ua_device_type', 'ua_client_type',
       'ua_os', 'ua_client_name', 'total_watchtime', 'rutube_video_id',
       'viewer_uid', 'title', 'category', 'duration', 'author_id'],
      dtype='object')

## Собираем таблицу по всем видео

Сгруппируем нашу таблицу по video_id и посчитаем количество просмотров для каждого видео

In [15]:
df = data.groupby(['rutube_video_id', 'title', 'category', 'duration', 'author_id']).size().reset_index(name='view_count')

In [16]:
df.head()

,rutube_video_id,title,category,duration,author_id,view_count
0,video_0,Киндерлинская пещера. Природный парк Зилим. Ба...,Разное,369105,1101320,1
1,video_1,"Запрет есть, наказания нет: суть ограничений н...",Телепередачи,266449,1008828,1
2,video_10,Разлука Орхидеи и Повелителя демонов- 1 сезон ...,Сериалы,2551733,1093191,20
3,video_100,Рок-фестиваль YLЕТАЙ-2023 / Протесты в Париже ...,Телепередачи,3956462,1016111,1
4,video_1000,Caliber gameplay.Калибр.Нужно больше навыков.,Видеоигры,6188011,1122425,1


In [17]:
df = df.sort_values(by='view_count', ascending=False)

In [18]:
df.reset_index()

,index,rutube_video_id,title,category,duration,author_id,view_count
0,351711,video_96775,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257,107378
1,317076,video_54649,"Новая Битва экстрасенсов, 24 сезон, 13 выпуск",Телепередачи,5642240,1009257,104141
2,91034,video_211211,"Новая Битва экстрасенсов, 24 сезон, 12 выпуск",Телепередачи,5450640,1009257,103911
3,176659,video_316010,"Новая Битва экстрасенсов, 24 сезон, 14 выпуск....",Телепередачи,5917000,1009257,64885
4,12984,video_115806,"Выжить в Дубае. Возвращение, 1 выпуск",Телепередачи,6743000,1009257,61151
...,...,...,...,...,...,...,...
354362,155452,video_290141,Евгений Коновалов - Любовник и Любовница,Музыка,218552,1023134,1
354363,155455,video_290146,Kim Taehyung BTS - Lose,Развлечения,285519,1015508,1
354364,155456,video_290147,КВН 2017 Высшая лига Первая 1/8 (26.02.2017),Юмор,6646804,1004166,1
354365,155459,video_29015,А как вам такие идеи из остатков ткани? :) 5 И...,Разное,1181107,1048898,1


In [19]:
df = df.reset_index(drop=True)

In [20]:
df.head()

,rutube_video_id,title,category,duration,author_id,view_count
0,video_96775,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257,107378
1,video_54649,"Новая Битва экстрасенсов, 24 сезон, 13 выпуск",Телепередачи,5642240,1009257,104141
2,video_211211,"Новая Битва экстрасенсов, 24 сезон, 12 выпуск",Телепередачи,5450640,1009257,103911
3,video_316010,"Новая Битва экстрасенсов, 24 сезон, 14 выпуск....",Телепередачи,5917000,1009257,64885
4,video_115806,"Выжить в Дубае. Возвращение, 1 выпуск",Телепередачи,6743000,1009257,61151


## Собираем информацию по возрасту и полу

### Соберем таблицу со статистикой по пользователям для каждого видео

In [21]:
df1 = pd.concat([df_all, df_train], axis = 0)
df2 = pd.read_csv('train_targets.csv')

In [22]:
import pandas as pd

# Загрузка данных (замените на свои пути к файлам)
video_data = pd.concat([df_all, df_train], axis=0)
viewer_data = pd.read_csv('train_targets.csv')

# Объединяем таблицы по столбцу viewer_uid
merged_data = pd.merge(video_data, viewer_data, on='viewer_uid', how='inner')

# Группируем данные по rutube_video_id
grouped = merged_data.groupby('rutube_video_id')

# Подсчитываем средний возраст и моду
result = grouped.agg(
    average_age=('age', 'mean'),
    mode_age=('age', lambda x: x.mode()[0]),
    total_viewers=('viewer_uid', 'count'),
    male_count=('sex', lambda x: (x == 'male').sum()),
    female_count=('sex', lambda x: (x == 'female').sum()),
    age_class_0=('age_class', lambda x: (x == 0).sum()),
    age_class_1=('age_class', lambda x: (x == 1).sum()),
    age_class_2=('age_class', lambda x: (x == 2).sum()),
    age_class_3=('age_class', lambda x: (x == 3).sum())
)

# Рассчитываем проценты
result['male_percentage'] = (result['male_count'] / result['total_viewers']) * 100
result['female_percentage'] = (result['female_count'] / result['total_viewers']) * 100
result['class_0_percentage'] = (result['age_class_0'] / result['total_viewers']) * 100
result['class_1_percentage'] = (result['age_class_1'] / result['total_viewers']) * 100
result['class_2_percentage'] = (result['age_class_2'] / result['total_viewers']) * 100
result['class_3_percentage'] = (result['age_class_3'] / result['total_viewers']) * 100

result = result.round(2)

# Добавляем столбец с количеством пользователей, которое было учтено для расчета статистики
result['total_users_used'] = result['total_viewers']

# Оставляем только нужные колонки
final_result = result[['average_age', 'mode_age', 'male_percentage', 'female_percentage',
                       'class_0_percentage', 'class_1_percentage', 
                       'class_2_percentage', 'class_3_percentage', 'total_users_used']].reset_index()


### Добавляем столбцы с статистикой к собранной ранее таблице

In [23]:
data = pd.merge(df, final_result, on = 'rutube_video_id', how = 'left')

In [24]:
data['average_age'].isna().sum()

228119

In [25]:
data.head()

,rutube_video_id,title,category,duration,author_id,view_count,average_age,mode_age,male_percentage,female_percentage,class_0_percentage,class_1_percentage,class_2_percentage,class_3_percentage,total_users_used
0,video_96775,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257,107378,31.47,24.0,15.50,84.50,6.00,42.81,36.84,14.35,23700.0
1,video_54649,"Новая Битва экстрасенсов, 24 сезон, 13 выпуск",Телепередачи,5642240,1009257,104141,31.66,24.0,15.94,84.06,5.64,42.41,37.23,14.72,23255.0
2,video_211211,"Новая Битва экстрасенсов, 24 сезон, 12 выпуск",Телепередачи,5450640,1009257,103911,31.59,24.0,15.87,84.13,5.68,42.84,36.87,14.61,23070.0
3,video_316010,"Новая Битва экстрасенсов, 24 сезон, 14 выпуск....",Телепередачи,5917000,1009257,64885,31.86,24.0,16.10,83.90,5.07,41.65,38.24,15.04,14627.0
4,video_115806,"Выжить в Дубае. Возвращение, 1 выпуск",Телепередачи,6743000,1009257,61151,32.13,24.0,22.20,77.80,2.25,42.80,42.09,12.86,18853.0


## Вычисление значений возраста и пола для Строк с пропусками

### Добавим эмбеддинги к нашей таблице

In [28]:
df_emb = pd.read_csv('all_emb_tiny.csv')

In [29]:
df_emb = df_emb[['rutube_video_id', 'embeddings']]

In [30]:
def parse_embedding(embedding_str):
    return np.fromstring(embedding_str.strip("[]"), sep=',')

In [31]:
# Примените функцию к столбцу с эмбеддингами
df_emb['embeddings'] = df_emb['embeddings'].apply(parse_embedding)

In [32]:
df_emb.head()

,rutube_video_id,embeddings
0,video_185549,"[0.34006547927856445, -0.8857478499412537, 0.2..."
1,video_111035,"[0.45694699883461, 0.48634690046310425, 0.0973..."
2,video_476517,"[-0.576312780380249, -0.5698351860046387, -0.0..."
3,video_157198,"[-0.5488875508308411, -0.25806984305381775, -0..."
4,video_289824,"[0.2059473991394043, -0.5128248333930969, 0.07..."


In [33]:
df = pd.merge(data, df_emb, on='rutube_video_id', how='left')

In [34]:
df.head()

,rutube_video_id,title,category,duration,author_id,view_count,average_age,mode_age,male_percentage,female_percentage,class_0_percentage,class_1_percentage,class_2_percentage,class_3_percentage,total_users_used,embeddings
0,video_96775,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257,107378,31.47,24.0,15.50,84.50,6.00,42.81,36.84,14.35,23700.0,"[0.06221436709165573, -0.3273929953575134, 0.2..."
1,video_54649,"Новая Битва экстрасенсов, 24 сезон, 13 выпуск",Телепередачи,5642240,1009257,104141,31.66,24.0,15.94,84.06,5.64,42.41,37.23,14.72,23255.0,"[-0.15979868173599243, -0.4690707325935364, 0...."
2,video_211211,"Новая Битва экстрасенсов, 24 сезон, 12 выпуск",Телепередачи,5450640,1009257,103911,31.59,24.0,15.87,84.13,5.68,42.84,36.87,14.61,23070.0,"[0.050529394298791885, -0.45317909121513367, 0..."
3,video_316010,"Новая Битва экстрасенсов, 24 сезон, 14 выпуск....",Телепередачи,5917000,1009257,64885,31.86,24.0,16.10,83.90,5.07,41.65,38.24,15.04,14627.0,"[-0.3305223882198334, -0.5321187376976013, 0.2..."
4,video_115806,"Выжить в Дубае. Возвращение, 1 выпуск",Телепередачи,6743000,1009257,61151,32.13,24.0,22.20,77.80,2.25,42.80,42.09,12.86,18853.0,"[-0.5561398267745972, 0.294213205575943, -0.12..."


In [ ]:
#df.to_csv('video_info.csv')

### Решаем проблему с пропусками

In [39]:
df[df.isnull().any(axis=1)].shape

(228119, 16)

In [73]:
k = 100  # минимальное количество посмотревших пользователей для учета видео

# Предварительная фильтрация видео с достаточным количеством пользователей
filtered_df = df[df['total_users_used'] > k]

# Перебор видео с пропусками
for index, row in df[df.isnull().any(axis=1)].iterrows():
    # Фильтрация исходного видео из выборки
    filtered_indices = filtered_df[filtered_df.index != index].index
    
    # Вычисление косинусного сходства только с отфильтрованными видео
    if len(filtered_indices) > 0:
        similarities = cosine_similarity([row['embeddings']], filtered_df.loc[filtered_indices, 'embeddings'].tolist())[0]
        
        # Получение индекса самого похожего видео
        best_match_index = filtered_indices[np.argmax(similarities)]
        
        # Заполнение пропусков значениями из самого релевантного видео
        df.loc[index, ['average_age', 'mode_age', 'male_percentage',
                       'female_percentage', 'class_0_percentage', 'class_1_percentage',
                       'class_2_percentage', 'class_3_percentage']] = df.loc[best_match_index, 
                       ['average_age', 'mode_age', 'male_percentage', 
                        'female_percentage', 'class_0_percentage', 'class_1_percentage',
                        'class_2_percentage', 'class_3_percentage']]

## Сохраняем итоговый датасет

In [74]:
df.to_csv('video_info.csv')